# How to Make a Decorator

[![DOI](https://zenodo.org/badge/214871831.svg)](https://zenodo.org/badge/latestdoi/214871831)

In [1]:
r"""markdown
    TITLE   :  Decorator Reference
    AUTHOR  :  Nathaniel Starkman
    PROJECT :  2019-10-22 dotAstronomy Plotting Workshop
"""

__author__ = 'Nathaniel Starkman'
__version__ = "Oct 22, 2019"

This notebook will be a walkthrough towards building a good decorator. We will do a lot of things wrong before getting them right.

Decorators are a powerful way to augment functions, and even classes. With a decorator we can alter the input or output of a function, and even edit the properties of a function. If you want a primer on decorators, [this is a good article](https://realpython.com/primer-on-python-decorators/).

In this notebook we will build up to a decorator that can edit the input and output of a function, will inherit the signature, annotations, and docstring of the decorated function, and can modify the inherited docstring. Decorators that properly modify the signature and annotations are a fair bit more challenging and not covered here.

<br><br>

- - - 
- - - 

<br><br>

# Preamble


In this section we do the necessary setup.

## Imports

Time for a shameless plug. I got fed up endlessly rewriting the same code and have started collecting them into a package I cleverly called [`astroPHD`](https://github.com/nstarman/astroPHD). Amongst many things, it has some useful ipython and import utilities.

In [2]:
from astroPHD import ipython
ipython.run_imports(base=True)

## General
import functools
import inspect

## Custom

## Project-Specific

set autoreload to 1
base_imports:
    Base: os, sys, time, pdb, warnings,
          numpy -> np, scipy,
          tqdm_notebook -> tqdm
    Logging: .LogFile
    Misc: ObjDict
    IPython: display, Latex, Markdown, set_trace,
             printmd, printMD, printltx, printLaTeX,
             configure_matplotlib,
             set_autoreload, aimport,
             run_imports, import_from_file,



# Functions


## Test Function

Decorators need a function to decorate. Here we define a very generic function with all 5 different types of arguments:

1. arguments
2. defaulted arguments
3. variable arguments,
4. keyword-only arguments
5. variable keyword arguments

In [3]:
def function(x: '1st arg', y: '2nd arg', a: '1st defarg'=10, b=11, *args: 'args',
             k: '1st kwonly'='one', l: '2nd kwonly'='two', **kw: 'kwargs') -> 'return_':
    '''function for testing decoration
    This function has all 5 different types of arguments:
        1) arguments, 2) defaulted arguments, 3) variable arguments,
        4) keyword-only arguments, 5) variable keyword arguments
    '''
    return x, y, a, b, args, k, l, kw
# /def

Checking the properties of the function,

In [4]:
function?

Signature:
function(
    x: '1st arg',
    y: '2nd arg',
    a: '1st defarg' = 10,
    b=11,
    *args: 'args',
    k: '1st kwonly' = 'one',
    l: '2nd kwonly' = 'two',
    **kw: 'kwargs',
) -> 'return_'
Docstring:
function for testing decoration
This function has all 5 different types of arguments:
    1) arguments, 2) defaulted arguments, 3) variable arguments,
    4) keyword-only arguments, 5) variable keyword arguments
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-3-151961bc205e>
Type:      function


This function, since it is the base definition of the function, has the correct location, name, signature, and docstring.

<br><br>

- - - 
- - - 

# Simple Decorators

Decorators can be constructed in a few different ways. We will start with the simplest method -- functions inside function.

The general construcution is, where `wrapper` controls how `function` is implemented. `decorator` creates the `wrapper` instance and applies it to the function. 

```python
def decorator(function):

    def wrapper():
        # do stuff here
        function()
        # and here
    return
return wrapper
```


## Base Python

These are the simplest decorators, using no imported packages.


There are two ways to apply a decorator. The first, by calling the function, is the most obvious and is very useful for modifying existing functions. The second, using the 'pie' syntax, is great when defining new functions.

In [5]:
def simple_decorator(function):
    '''simple decorator
    made without any extra packages
    '''
    def wrapper(*args, **kw):
        """accepts any input and passes to the function"""
        print("Pre function call")
        print(function(*args, **kw))
        print("Post function call")
        return
    # /def
    return wrapper
# /def

### Function Decorator Method

This is the first of the two decorator methods -- through function calling. We will make a new function, decorated by `simple_decorator`, simply by calling `simple_decorator(function)`.

In [6]:
simple_function = simple_decorator(function)

Now calling the function

In [7]:
simple_function(1, 2, k='1st keyword')

Pre function call
(1, 2, 10, 11, (), '1st keyword', 'two', {})
Post function call


Succcess! The print statements were effective and the function was called with the modified inputs and used all the expected default values.

#### Introspection

In [8]:
simple_function?

Signature: simple_function(*args, **kw)
Docstring: accepts any input and passes to the function
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-5-d37d3cd62838>
Type:      function


Now for the bad news.

When we compare this to `function?` (above), this decorator has lost most of the information. The function now has a different signature and the docstring from the wrapper. This is not great.

<br>

### Decorator Syntax Method

before we go and make a better decorator, this is how to use the 'pie' decorator syntax.

In [9]:
@simple_decorator
def simple_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
    '''function for testing decoration
    This function has all 5 different types of arguments
    '''
    return x, y, a, b, args, k, l, kw
# /def

In [10]:
simple_dec_function(1, 2)

Pre function call
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


#### Introspection

In [11]:
simple_dec_function?

Signature: simple_dec_function(*args, **kw)
Docstring: accepts any input and passes to the function
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-5-d37d3cd62838>
Type:      function


Again, the introspection fails quite badly. We can do better.

<br><br>

- - - 

## Functools Decorators

The easiest way to make a better decorator is with the built-in package `functools`. Using the function `wraps` from the `functools` package, our simple decorators can now preserve function signatures and docstrings.

Now, the decorator construction syntax is

```python
def decorator(function):

    @functools.wraps(function)
    def wrapper():
        # do stuff here
        function()
        # and here
    return
return wrapper
```

Yes, we now have a decorator inside our decorator-making function. If this were a tutorial on decorators and decorator factories, the decorator nesting would have only just begun.

Let's see this decorator in action.

### Function Decorator Method

Again, starting with the function calling method,

In [12]:
def ftw_decorator(function):
    '''FuncTools.Wraps (ftw) decorator
    '''
    @functools.wraps(function)
    def wrapper(*args, **kw):
        print("Pre function call")
        print(function(*args, **kw))
        print("Post function call")
        return
    # /def
    return wrapper
# /def


In [13]:
ftw_function = ftw_decorator(function)

ftw_function(1, 2)

Pre function call
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


#### Introspection

In [14]:
# introspect_function(ftw_function)
ftw_function??

Signature:
ftw_function(
    x: '1st arg',
    y: '2nd arg',
    a: '1st defarg' = 10,
    b=11,
    *args: 'args',
    k: '1st kwonly' = 'one',
    l: '2nd kwonly' = 'two',
    **kw: 'kwargs',
) -> 'return_'
Source:   
def function(x: '1st arg', y: '2nd arg', a: '1st defarg'=10, b=11, *args: 'args',
             k: '1st kwonly'='one', l: '2nd kwonly'='two', **kw: 'kwargs') -> 'return_':
    '''function for testing decoration
    This function has all 5 different types of arguments:
        1) arguments, 2) defaulted arguments, 3) variable arguments,
        4) keyword-only arguments, 5) variable keyword arguments
    '''
    return x, y, a, b, args, k, l, kw
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-3-151961bc205e>
Type:      function


<span style='font-size:20px;font-weight:650'>
    The new function matches the original function!
</span>


![](figures/success.png)



### Decorator Syntax Method

In [15]:
@ftw_decorator
def ftw_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
    '''function for testing decoration
    This function has all 5 different types of arguments
    '''
    return x, y, a, b, args, k, l, kw
# /def

In [16]:
ftw_dec_function(1, 2)

Pre function call
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


#### Introspection

In [17]:
# introspect_function(ftw_dec_function)
ftw_dec_function??

Signature: ftw_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw)
Source:   
@ftw_decorator
def ftw_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
    '''function for testing decoration
    This function has all 5 different types of arguments
    '''
    return x, y, a, b, args, k, l, kw
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-15-9c2fa596773f>
Type:      function


Again, the decorator preserves the signature and docstring. Great!

<br><br>

- - - 

## External Packages

Everything so far was with base Python. There are (at least) two mature packages for making decorators: `decorator` and `wraps`. I don't detail how to use them here, thogh this could change if I discover features pertinent to my work that cannot be easily accomplished with base Python.

If you are looking to make unified classmethod and staticmethod and general function decorators, check out `wraps` as I have not tested these decorators for full compatibility. For generic function creation, the `decorator` package offers a powerful feature called `FunctionMaker` (note this function is not compatible with Signature objects, otherwise I would use it here). Astropy offers a `make_function_with_signature` that is somewhat similar to `FunctionMaker`.

<br><br><br><br>

- - - 
- - - 

# Decorators Which Add (kw)Arguments


## Base+functools+inspect Python

In [101]:
def simple_addkw_decorator(function):
    '''simple decorator
    made without any extra packages
    tris to modify docstrings & Signatures
    '''

    @functools.wraps(function)
    def wrapper(*args, added_kw='t', **kw):
        print("Pre function call")
        print('added_kw:', added_kw)
        print(function(*args, **kw))
        print("Post function call")
        return

    # /def
    return wrapper
# /def

In [103]:
simple_addkw_function = simple_addkw_decorator(function)

simple_addkw_function(1, 2, added_kw='t->u')

Pre function call
added_kw: t->u
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


In [104]:
simple_addkw_function??

Signature:
simple_addkw_function(
    x: '1st arg',
    y: '2nd arg',
    a: '1st defarg' = 10,
    b=11,
    *args: 'args',
    k: '1st kwonly' = 'one',
    l: '2nd kwonly' = 'two',
    **kw: 'kwargs',
) -> 'return_'
Source:   
def function(x: '1st arg', y: '2nd arg', a: '1st defarg'=10, b=11, *args: 'args',
             k: '1st kwonly'='one', l: '2nd kwonly'='two', **kw: 'kwargs') -> 'return_':
    '''function for testing decoration
    This function has all 5 different types of arguments:
        1) arguments, 2) defaulted arguments, 3) variable arguments,
        4) keyword-only arguments, 5) variable keyword arguments
    '''
    return x, y, a, b, args, k, l, kw
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-3-151961bc205e>
Type:      function


<br>
Unfortunately, introspection gives us no clue that a keyword argument has been added.
To address this shortcoming, we will next make decorators that can edit the docstrings of the wrapped functions.

<br><br><br><br>

- - - 
- - - 

# Docstring-Modifying Decorators

The goal is to make a decorator that modifies the docstring of the decorated function, so that we can document what the decorator is doing.

## Base Python (w/ functools)

In [61]:
def simple_mod_decorator(function):
    '''simple decorator
    made without any extra packages
    permits any modification to the docstring
    
    Parameters
    ----------
    function: Function
        function to be decorated
    doc_func: Function, None  (default None)
        function to edit the docstring
        signature: f(__doc__)
    
    Returns
    -------
    wrapper: Function
        wrapped function
    '''
    @functools.wraps(function)
    def wrapper(*args, **kw):
        print("Pre function call")
        print(function(*args, **kw))
        print("Post function call")
        return
    # /def

    # editing docstring
    if function.__doc__ is not None:
        wrapper.__doc__ = function.__doc__ + ("\n"
                                              "Decorator\n"
                                              "-------\n"
                                              "prints information about function")

    return wrapper
# /def

Making sure the decorated function works as originally coded

In [62]:
simple_mod_function = simple_mod_decorator(function)

simple_mod_function(1, 2)

Pre function call
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


Let's see how the docstring turned out.

In [63]:
simple_mod_function?

Signature:
simple_mod_function(
    x: '1st arg',
    y: '2nd arg',
    a: '1st defarg' = 10,
    b=11,
    *args: 'args',
    k: '1st kwonly' = 'one',
    l: '2nd kwonly' = 'two',
    **kw: 'kwargs',
) -> 'return_'
Docstring:
function for testing decoration
    This function has all 5 different types of arguments:
        1) arguments, 2) defaulted arguments, 3) variable arguments,
        4) keyword-only arguments, 5) variable keyword arguments
    
Decorator
-------
prints information about function
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-3-151961bc205e>
Type:      function


And as simple as that, we have modified the function's docstring

### Decorator Syntax Method

Just checking.  This should work since we are not doing anything too strange.

In [68]:
@simple_mod_decorator
def simple_mod_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
    '''function for testing decoration
    This function has all 5 different types of arguments
    '''
    return x, y, a, b, args, k, l, kw
# /def

In [69]:
simple_mod_dec_function(1, 2)

Pre function call
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


In [70]:
simple_mod_function?

Signature:
simple_mod_function(
    x: '1st arg',
    y: '2nd arg',
    a: '1st defarg' = 10,
    b=11,
    *args: 'args',
    k: '1st kwonly' = 'one',
    l: '2nd kwonly' = 'two',
    **kw: 'kwargs',
) -> 'return_'
Docstring:
function for testing decoration
    This function has all 5 different types of arguments:
        1) arguments, 2) defaulted arguments, 3) variable arguments,
        4) keyword-only arguments, 5) variable keyword arguments
    
Decorator
-------
prints information about function
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-3-151961bc205e>
Type:      function


Indeed, it works exactly as expected.

## Arbitrary Modifications to the Docstring

This is a cool side note and only introduced here to motivate a different paradigm for constructing decorators.

Instead of pre-prescibing a modification to the docstring, we are trying to allow arbitrary modifications using
an external function. When we call the decorator, we should also pass the docstring editor.

Let's see how this works.

In [75]:
def simple_mod_decorator(function, doc_func=None):
    '''simple decorator
    made without any extra packages
    permits any modification to the docstring
    
    Parameters
    ----------
    function: Function
        function to be decorated
    doc_func: Function, None  (default None)
        function to edit the docstring
        signature: f(__doc__)
    
    Returns
    -------
    wrapper: Function
        wrapped function
    '''
    @functools.wraps(function)
    def wrapper(*args, **kw):
        print("Pre function call")
        print(function(*args, **kw))
        print("Post function call")
        return
    # /def

    # editing docstring
    doc_func = doc_func if doc_func is not None else (lambda x: x)
    wrapper.__doc__ = doc_func(function.__doc__)

    return wrapper
# /def

In [76]:
simple_mod_function = simple_mod_decorator(function, doc_func=lambda x: x + "\nI've added this information")

simple_mod_function(1, 2)

Pre function call
(1, 2, 10, 11, (), 'one', 'two', {})
Post function call


In [77]:
simple_mod_function?

Signature:
simple_mod_function(
    x: '1st arg',
    y: '2nd arg',
    a: '1st defarg' = 10,
    b=11,
    *args: 'args',
    k: '1st kwonly' = 'one',
    l: '2nd kwonly' = 'two',
    **kw: 'kwargs',
) -> 'return_'
Docstring:
function for testing decoration
    This function has all 5 different types of arguments:
        1) arguments, 2) defaulted arguments, 3) variable arguments,
        4) keyword-only arguments, 5) variable keyword arguments
    
I've added this information
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-3-151961bc205e>
Type:      function


It seems to work great!

Now let's make sure that this decorator works when we use the 'pie' syntax.

### Decorator Syntax Method

In [277]:
try:
    @simple_mod_decorator(doc_func="\nI've added this information")
    def simple_mod_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
        '''function for testing decoration
        This function has all 5 different types of arguments
        '''
        return x, y, a, b, args, k, l, kw
    # /def

except TypeError as e:
    print(e)

simple_mod_decorator() missing 1 required positional argument: 'function'


Uh Oh!

This method fails because it needs the function as an argument. Passing only the `doc_func` wasn't enough.

The next section introduces a fix.

<br>

## Fix using Class-based Decorators



There are fixes that allow us to still use functions as decorators. But now is the perfect opportunity to introduce a more powerful way of making decorators...  CLASSES!

In [79]:
class Simple_Mod_Decorator():
    """A class-based implementation of simple_mod_decorator
    """

    def __new__(cls, func=None, doc_func=None):
        """
        this is a quick and dirty method for class-based decorator creation
        it is generically better to do this with a classmethod like
        @classmethod
        as_decorator(cls, func=None, ...):
            all the same code as here
        """
        # make instance
        self = super().__new__(cls)
        
        # wrapper control:
        if func is not None:  # this will return a wrapped function
            # pass all arguments and kwargs to init
            # since __init__ is will not be called
            self.__init__(**{k: v for k, v in locals().items()
                             if k not in ('cls', 'self', '__class__')})
            return self(func)
        else:  # this will return a function wrapper
            # for when using as a @decorator
            # __init__ will be automatically called after this
            return self
    # /def

    def __init__(self, func=None, doc_func=None, doc_func_args=[], doc_func_kwargs={}):
        """
        these are stored to be used inside of __call__
        they are not normally passed to the wrapped_function
        """
        # default docstring function 
        doc_func = doc_func if doc_func is not None else (lambda x: x)

        # store all values passed to __init__
        for k, v in locals().items():
            setattr(self, k, v)
    # /def

    def __call__(self, wrapped_function):
        """construct a function wrapper
        """
        @functools.wraps(wrapped_function)
        def wrapper(*func_args, **func_kwargs):
            return_ = wrapped_function(*func_args, **func_kwargs)
            return return_
        # /def

        # docstring
        if wrapped_function.__doc__ is not None:
            wrapper.__doc__ = self.doc_func(wrapped_function.__doc__, *self.doc_func_args, **self.doc_func_kwargs)

        # storing extra info
        wrapper.doc_func = self.doc_func
        
        return wrapper
    # /def
# /class

In [80]:
@Simple_Mod_Decorator(doc_func=lambda x: x + "\nI've added this information")
def simple_mod_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
    '''function for testing decoration
    This function has all 5 different types of arguments
    '''
    return x, y, a, b, args, k, l, kw
# /def

In [81]:
simple_mod_dec_function(1, 2)

simple_mod_dec_function.doc_func

(1, 2, 10, 11, (), 'one', 'two', {})

<function __main__.<lambda>(x)>

In [86]:
simple_mod_dec_function?

Signature: simple_mod_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw)
Docstring:
function for testing decoration
    This function has all 5 different types of arguments
    
I've added this information
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-80-4dd1abe7d9d1>
Type:      function


As can be seen here, this is great

<br><br><br><br>
<br><br><br><br>

- - - 
- - - 

# Combining Everything

This is a simple impolementation that combines everything thus far

Making a generic base for decorators and decorator factories that can 

In [276]:
class DecoratorClassBase():
    """
    """
    
    @staticmethod
    def _doc_func(docstring):
        return docstring

    def __new__(cls, func=None):
        """
        this is a quick and dirty method for class-based decorator creation
        it is generically better to do this with a classmethod like
        @classmethod
        as_decorator(cls, func=None, ...):
            all the same code as here
        """
        # make instance
        self = super().__new__(cls)
        
        # wrapper control:
        if func is not None:  # this will return a wrapped function
            # pass all arguments and kwargs to init
            # since __init__ is will not be called
            self.__init__(**{k: v for k, v in locals().items()
                             if k not in ('cls', 'self', '__class__')})
            return self(func)
        else:  # this will return a function wrapper
            # for when using as a @decorator
            # __init__ will be automatically called after this
            return self
    # /def

    def __init__(self, func=None):
        """
        these are stored to be used inside of __call__
        they are not normally passed to the wrapped_function
        """

        # store all values passed to __init__
        for k, v in locals().items():
            setattr(self, k, v)
    # /def

    def edit_docstring(self, wrapper):
        """blank call
        """

        # docstring
        if wrapper.__doc__ is not None:
            wrapper.__doc__ = self._doc_func(wrapper.__doc__)

        # storing extra info
        wrapper._doc_func = self._doc_func
        
        return wrapper
    # /def
# /class

In [271]:
class NewDecorator(DecoratorClassBase):
    """this is a new decorator
    all we need to do is inherit from DecoratorClassBase,
    define a _doc_func and make our wrapper function
    """

    @staticmethod
    def _doc_func(docstring):
        return docstring + ("\nDecorator\n"
                           "-------\n"
                            "added_kw: str\n\tadd a key-word argument")
    
    def __call__(self, wrapped_function):
        """construct a function wrapper
        """
        @functools.wraps(wrapped_function)
        def wrapper(*func_args, added_kw='added_kw', **func_kwargs):
            print(added_kw)
            return_ = wrapped_function(*func_args, **func_kwargs)
            return return_
        # /def

        return self.edit_docstring(wrapper)
    # /def


In [272]:
@NewDecorator
def simple_mod_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw):
    '''function for testing decoration
    This function has all 5 different types of arguments
    '''
    return x, y, a, b, args, k, l, kw
# /def

In [273]:
simple_mod_dec_function(1, 2)

simple_mod_dec_function._doc_func

added_kw


(1, 2, 10, 11, (), 'one', 'two', {})

<function __main__.NewDecorator._doc_func(docstring)>

In [274]:
simple_mod_dec_function?

Signature: simple_mod_dec_function(x, y, a=10, b=11, *args, k='one', l='two', **kw)
Docstring:
function for testing decoration
    This function has all 5 different types of arguments
    
Decorator
-------
added_kw: str
        add a key-word argument
File:      ~/Documents/Conferences/2019-10-22 dotAstronomy/presentation/notebooks/matplotlib/decorator_plotting/<ipython-input-272-402682b5e370>
Type:      function


<br><br><br><br><br><br>

- - - 
- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>